5장에서는 여러 개의 컨테이너를 동시에 실행해 컨테이너끼리 통신하는 방법을 배운다. 컨테이너 간의 통신을 위한 네트워크를 만드는 방법을 새롭게 배우지만, 그 외의 내용은 4장에서 배운 내용을 응용한 것이다.

# 01. 워드프레스 구축

컨테이너에 워드프레스 사이트를 구축해보자. 워드프레스를 컨테이너에서 동작하게 하려면 여러 개의 컨테이너를 실행하고 가상 네트워크를 통해 컨테이너끼리 통신할 수 있어야 한다. 이번 절에서는 워드프레스 사이트를 구축하는 과정 및 이 과정에 쓰이는 명령어에 대해 배운다.

## 워드프레스 사이트 구성 및 구축

워드프레스는 웹사이트를 만들기 위한 소프트웨어로, 서버에 설치해 사용한다.

<img src="./image/d05-01.png" width="60%"></img>

## 도커 네트워크 생성/삭제

**워드프레스는 워드프레스 컨테이너와 MySQL 컨테이너로 구성된다.**

워드 프레스는 간단히 말해 블로그 생성 도구와 같은 것으로, 웹 사이트 작성자가 작성한 내용을 데이터 베이스에 저장하고, 웹사이트 열람자의 요청에 따라 웹페이지를 보여준다. 즉, 프로그램이 MySQL에 저장된 데이터를 읽고 쓸 수 있어야 하기 때문에 두 컨테이너가 연결돼 있어야 한다.

### 도커 네트워크를 생성하는 커맨드

In [ ]:
docker network create 네트워크_이름

### 도커 네트워크를 삭제하는 커맨드

In [ ]:
docker network rm 네트워크_이름

In [ ]:
docker network ls

### 그 외 도커 네트워크 관련 커맨드

<img src="./image/d05-02.png" width="60%"></img>

## MySQL 컨테이너 실행 시에 필요한 옵션과 인자

자주 사용하는 커맨드 예

In [ ]:
docker run --name 컨테이너_이름 -dit --net=네트워크_이름 \
  -e MYSQL_ROOT_PASSWORD=MySQL_루트_패스워드 \
  -e MYSQL_DATABASE=데이터베이스_이름 \
  -e MYSQL_USER=MySQL_사용자이름 \
  -e MYSQL_PASSWORD=MySQL_패스워드 \
  mysql:8.0 \
  --character-set-server=문자_인코딩 \
  --collation-server=정렬_순서 \
  --default-authentication-plugin=인증_방식

### 사용된 옵션

<img src="./image/d05-03.png" width="60%"></img>

패스워드는 **루트 패스워드**와 일반 **사용자 패스워드** 두 가지를 설정한다.  
루트는 모든 권한을 가진 사용자로, 매번 루트 사용자로 접속할 경우 보안 측면에서 문제가 생기기 때문에 제한된 권한을 가진 일반 사용자로 전환하는 것이 일반적이다.

### 사용된 인자

<img src="./image/d05-04.png" width="60%"></img>

세 번째에 나온 인증 방식은 MySQL이 MySQL5(최신 버전 5.7)에서 MySQL8(8.0)로 넘어오면서 **외부 소프트웨어가 MySQL에 접속하는 인증 방식을 바꾸었기** 때문이다. 이 새로운 인증 방식은 아직 지원하지 않는 소프트웨어가 많다.

따라서 여기서는 접속에 문제가 없도록 인증 방식을 예전 방식으로 되돌려 놓는다.

### 워드프레스 컨테이너 실행 시 필요한 옵션과 인자

In [ ]:
docker run --name 컨테이너_이름 -dit --net=네트워크_이름 -p 포트_설정 \
  -e WORDPRESS_DB_HOST=데이터베이스_컨테이너_이름 \
  -e WORDPRESS_DB_NAME=데이터베이스_이름 \
  -e WORDPRESS_DB_USER=데이터베이스_사용자_이름 \
  -e WORDPRESS_DB_PASSWORD=데이터베이스_패스워드 \
  wordpress

<img src="./image/d05-05.png" width="60%"></img>
<img src="./image/d05-06.png" width="60%"></img>

다시 말해 워드프레스는 데이터베이스를 사용하는 소프트웨어이므로 연동될 데이터베이스에 대한 정보를 설정한 것이다.

지금까지 설정한 정보는 물론 MySQL 컨테이너의 설정값과 동일한 값이어야 한다. 값이 다르면 연동이 잘 되지 않으며 워드프레스도 사용할 수 없다.

# 02. 워드프레스 및 MySQL 컨테이너 생성과 연동

## 이번 절의 실습 내용과 사용할 커맨드

워드프레스 컨테이너와 MySQL 컨테이너를 생성하고 실행한다. MySQL 컨테이너를 먼저 생성하는 것과 네트워크를 생성하는 것을 잊지 않도록 한다.

<img src="./image/d05-07-1.png" width="30%"></img>

<img src="./image/d05-07.png" width="60%"></img>
<img src="./image/d05-08.png" width="45%"></img>

### 커맨드에서 사용할 옵션, 대상, 인자

네트워크 생성

In [ ]:
docker network create 네트워크_이름

MySQL 컨테이너 생성 및 실행

In [ ]:
docker run --name 컨테이너_이름 -dit --net=네트워크_이름 \
  -e MYSQL_ROOT_PASSWORD=MySQL_루트_패스워드 \
  -e MYSQL_DATABASE=데이터베이스_이름 \
  -e MYSQL_USER=MySQL_사용자이름 \
  -e MYSQL_PASSWORD=MySQL_패스워드 \
  mysql:8.0 \
  --character-set-server=문자_인코딩 \
  --collation-server=정렬_순서 \
  --default-authentication-plugin=인증_방식

워드프레스 컨테이너 생성 및 실행

In [ ]:
docker run --name 컨테이너_이름 -dit --net=네트워크_이름 -p 포트_설정 \
  -e WORDPRESS_DB_HOST=데이터베이스_컨테이너_이름 \
  -e WORDPRESS_DB_NAME=데이터베이스_이름 \
  -e WORDPRESS_DB_USER=데이터베이스_사용자_이름 \
  -e WORDPRESS_DB_PASSWORD=데이터베이스_패스워드 \
  wordpress

## 워드프레스와 MySQL 컨테이너 생성 및 실행

#### 단계 1-- network create 커맨드를 사용해 네트워크 생성

In [1]:
!docker network create wordpress000net1

838feaa399f09f53e647338f1c9976fd8b1ebb816ca55b063c8eae51948f1cc1


In [2]:
!docker network ls

NETWORK ID     NAME               DRIVER    SCOPE
c5e98858cb65   bridge             bridge    local
db7534889888   host               host      local
3f3d252b73eb   none               null      local
838feaa399f0   wordpress000net1   bridge    local


#### 단계 2 run 커맨드로 MySQL 컨테이너 생성 및 실행

In [3]:
!docker run --name mysql000ex11 -dit --net=wordpress000net1 \
  -e MYSQL_ROOT_PASSWORD=myrootpass \
  -e MYSQL_DATABASE=wordpress000db \
  -e MYSQL_USER=wordpress000kun \
  -e MYSQL_PASSWORD=wkunpass \
  mysql:8.0 \
  --character-set-server=utf8mb4 \
  --collation-server=utf8mb4_unicode_ci \
  --default-authentication-plugin=mysql_native_password

b07a434f6623d9b2d93feed57bbf40604361574fde924e46c0cbaaa0e721cb39


Unable to find image 'mysql:8.0' locally
8.0: Pulling from library/mysql
d9a40b27c30f: Pulling fs layer
7d8ccc83ce1d: Pulling fs layer
f98ae2ef29c4: Pulling fs layer
2e1aa9898aa1: Pulling fs layer
d0229fb4f83e: Pulling fs layer
bdfea4c359d4: Pulling fs layer
d6fe59aa1e5c: Pulling fs layer
7336dce1c47f: Pulling fs layer
1c71c9d185d1: Pulling fs layer
e63d4802c63b: Pulling fs layer
383f329967ab: Pulling fs layer
2e1aa9898aa1: Waiting
d0229fb4f83e: Waiting
bdfea4c359d4: Waiting
d6fe59aa1e5c: Waiting
7336dce1c47f: Waiting
e63d4802c63b: Waiting
1c71c9d185d1: Waiting
383f329967ab: Waiting
7d8ccc83ce1d: Verifying Checksum
7d8ccc83ce1d: Download complete
f98ae2ef29c4: Verifying Checksum
f98ae2ef29c4: Download complete
d0229fb4f83e: Verifying Checksum
d0229fb4f83e: Download complete
2e1aa9898aa1: Verifying Checksum
2e1aa9898aa1: Download complete
bdfea4c359d4: Verifying Checksum
bdfea4c359d4: Download complete
7336dce1c47f: Verifying Checksum
7336dce1c47f: Download complete
1c71c9d185d1: Verify

#### 단계 3 'run' 커맨드로 WordPress 컨테이너를 생성 및 실행하기

In [4]:
!docker run --name wordpress000ex12 -dit --net=wordpress000net1 -p 8085:80 \
  -e WORDPRESS_DB_HOST=mysql000ex11 \
  -e WORDPRESS_DB_NAME=wordpress000db \
  -e WORDPRESS_DB_USER=wordpress000kun \
  -e WORDPRESS_DB_PASSWORD=wkunpass \
  wordpress

7ceea97e276bdd0ad0b7c5659ad69bd205526a6491b312292cabdbdedc07f47d


Unable to find image 'wordpress:latest' locally
latest: Pulling from library/wordpress
e4fff0779e6d: Pulling fs layer
ebe65c9579cf: Pulling fs layer
73fb9bdf2456: Pulling fs layer
029db5f1c17f: Pulling fs layer
364fd66af37d: Pulling fs layer
de55dbd5d220: Pulling fs layer
18b6e8540b90: Pulling fs layer
8e94b687cf8e: Pulling fs layer
67c4fe835099: Pulling fs layer
69122bd6c4de: Pulling fs layer
399b24a49abd: Pulling fs layer
126a489d1344: Pulling fs layer
a1fef752f3f6: Pulling fs layer
37d8441a5a6f: Pulling fs layer
56bbd85fd130: Pulling fs layer
2e44139c2607: Pulling fs layer
98d6faf3f1a7: Pulling fs layer
affb05935c5d: Pulling fs layer
1b7cfe87833c: Pulling fs layer
b9a329a4a02d: Pulling fs layer
85da70461387: Pulling fs layer
8e94b687cf8e: Waiting
029db5f1c17f: Waiting
67c4fe835099: Waiting
364fd66af37d: Waiting
69122bd6c4de: Waiting
de55dbd5d220: Waiting
399b24a49abd: Waiting
18b6e8540b90: Waiting
126a489d1344: Waiting
a1fef752f3f6: Waiting
37d8441a5a6f: Waiting
56bbd85fd130: Waitin

#### 단계 4 'ps' 커맨드로 컨테이너가 실행 중인지 확인하기

In [6]:
!docker ps

CONTAINER ID   IMAGE       COMMAND                   CREATED          STATUS         PORTS                  NAMES
7ceea97e276b   wordpress   "docker-entrypoint.s…"   9 minutes ago    Up 9 minutes   0.0.0.0:8085->80/tcp   wordpress000ex12
b07a434f6623   mysql:8.0   "docker-entrypoint.s…"   10 minutes ago   Up 9 minutes   3306/tcp, 33060/tcp    mysql000ex11


#### 단계 5 웹 브라우저를 통해 워드프레스에 접근해 확인

In [ ]:
# !http://localhost:8085

<img src="./image/d05-09.png" width="60%"></img>

#### 단계 6 뒷정리하기

컨테이너 종료

In [7]:
!docker stop wordpress000ex12
!docker stop mysql000ex11

wordpress000ex12
mysql000ex11


컨테이너 삭제

In [8]:
!docker rm wordpress000ex12
!docker rm mysql000ex11

wordpress000ex12
mysql000ex11


이미지 삭제

In [9]:
!docker image rm wordpress
!docker image rm mysql:8.0

Untagged: wordpress:latest
Untagged: wordpress@sha256:c029a137bec0b344fc1ced1980d12a82b93aae9105240afd4e3b1bf83d79a618
Deleted: sha256:e826d932809cc344b40bea6d0cefe9eb9c71c148d2991be8ecbe8eee31196b14
Deleted: sha256:604c559eb3dbd799cd6f734bc4ca84ccb729623df7a846527d3f9f2e17577816
Deleted: sha256:60c6580bd07585c3670e8b6619c5e0ab16c052578b670951c4eeebc9f6ab155e
Deleted: sha256:abb569aa0b609d104bb35368d78b44e531a7216bb1578bfdb509281ff4fd30ee
Deleted: sha256:474d56773d22f771210ea6b1ce60585eddaded9ece60ba82cc0089029d5e8f52
Deleted: sha256:5cca8027d2580898e68362b644ef3cd521ebf9622cde106bbe6065f70e61a96b
Deleted: sha256:53a798400e29b5322b4a15dd44e2c8ee090e58fcbc0207dbee6a5202c2b39d77
Deleted: sha256:c7a9ebaa1c23bb551ea89bb4505221df5b4fe1ca394629aca952e3a7a6a12aa7
Deleted: sha256:30e152dbc08add3c1651d114575cf30d9269cb1acf24875696b14b0b61498cc1
Deleted: sha256:484c23b88e399c88e5326717502441cb5914da031fa17881269c5d1329dbf699
Deleted: sha256:eeb4feacd2042d826502a7ae138ab80da69af01b9049cbf784455e5

In [10]:
!docker network rm wordpress000net1

wordpress000net1


# 03.명령어를 직접 작성하자

## 소프트웨어와 데이터베이스의 관계

워드프레스를 사용하려면 워드프레스 프로그램 외에도 아파치와 PHP 런타임, MySQL이 필요했다.  
워드프레스 외에도 **이러한 형식으로 구성되는 웹 시스템**이 많다.

**리눅스 + 웹 서버 + 프로그래밍 언어 런타임 + 데이터베이스**

'프로그램 본체 + 프로그램 런타임 + 웹서버' 컨테이너 + '데이터베이스' 컨테이너

**LAMP 스택**

<img src="./image/d05-10.png" width="60%"></img>

### 문제 1: MySQL 컨테이너의 옵션 값 채워넣기

MySQL run 명령어

<img src="./image/d05-11.png" width="60%"></img>

In [ ]:
docker run --name mysql000ex11 -dit \
  --net=wordpress000net1 \
  -e MYSQL_ROOT_PASSWORD=myrootpass \
  -e MYSQL_DATABASE=wordpress000db \
  -e MYSQL_USER=wordpress000kun \
  -e MYSQL_PASSWORD=wkunpass \
  mysql:8.0 \
  --character-set-server=utf8mb4 \
  --collation-server=utf8mb4_unicode_ci \
  --default-authentication-plugin=mysql_native_password

### 문제 2: 워드프레스 컨테이너의 옵션 값 채워넣기

워드프레스 run 명령어

<img src="./image/d05-12.png" width="60%"></img>
<img src="./image/d05-13.png" width="60%"></img>

In [ ]:
docker run --name wordpress000ex12 -dit \
  --net=wordpress000net1 -p 8085:80 \
  -e WORDPRESS_DB_HOST=mysql000ex11 \
  -e WORDPRESS_DB_NAME=wordpress000db \
  -e WORDPRESS_DB_USER=wordpress000kun \
  -e WORDPRESS_DB_PASSWORD=wkunpass \
  wordpress

# 04. 레드마인 및 MariaDB 컨테이너를 대상으로 연습하자

## 레드마인 및 MySQL 컨테이너 생성

<img src="./image/d05-14.png" width="30%"></img>

### 생성할 네트워크 및 컨테이너의 정보

<img src="./image/d05-15.png" width="30%"></img>

### 커맨드에서 사용할 옵션, 대상, 인자

네트워크 생성

In [11]:
!docker network create redmine000net2

a3620b565b35821183d5115979b9a95f6432d11e5cfa03ca93975e2cb0ba79b4


MySQL 컨테이너 생성 및 실행

In [13]:
!docker run --name mysql000ex13 -dit \
  --net=redmine000net2 \
  -e MYSQL_ROOT_PASSWORD=myrootpass \
  -e MYSQL_DATABASE=redmine000db \
  -e MYSQL_USER=redmine000kun \
  -e MYSQL_PASSWORD=rkunpass \
  mysql:8.0 \
  --character-set-server=utf8mb4 \
  --collation-server=utf8mb4_unicode_ci \
  --default-authentication-plugin=mysql_native_password

f5622a91d5a36d8729fac07c24c9d2e875ff2378ca21f8868a0afa3bac741720


Unable to find image 'mysql:8.0' locally
8.0: Pulling from library/mysql
d9a40b27c30f: Pulling fs layer
7d8ccc83ce1d: Pulling fs layer
f98ae2ef29c4: Pulling fs layer
2e1aa9898aa1: Pulling fs layer
d0229fb4f83e: Pulling fs layer
bdfea4c359d4: Pulling fs layer
d6fe59aa1e5c: Pulling fs layer
7336dce1c47f: Pulling fs layer
1c71c9d185d1: Pulling fs layer
e63d4802c63b: Pulling fs layer
383f329967ab: Pulling fs layer
2e1aa9898aa1: Waiting
e63d4802c63b: Waiting
d0229fb4f83e: Waiting
d6fe59aa1e5c: Waiting
7336dce1c47f: Waiting
1c71c9d185d1: Waiting
383f329967ab: Waiting
bdfea4c359d4: Waiting
7d8ccc83ce1d: Verifying Checksum
7d8ccc83ce1d: Download complete
f98ae2ef29c4: Verifying Checksum
f98ae2ef29c4: Download complete
d0229fb4f83e: Verifying Checksum
d0229fb4f83e: Download complete
2e1aa9898aa1: Verifying Checksum
2e1aa9898aa1: Download complete
bdfea4c359d4: Verifying Checksum
bdfea4c359d4: Download complete
7336dce1c47f: Download complete
1c71c9d185d1: Verifying Checksum
1c71c9d185d1: Downlo

<img src="./image/d05-16.png" width="50%"></img>

레드마인 컨테이너의 생성 및 실행

In [14]:
!docker run -dit --name redmine000ex14 \
  --network redmine000net2 -p 8086:3000 \
  -e REDMINE_DB_MYSQL=mysql000ex13 \
  -e REDMINE_DB_DATABASE=redmine000db \
  -e REDMINE_DB_USERNAME=redmine000kun \
  -e REDMINE_DB_PASSWORD=rkunpass \
  redmine

1d97bdb0bf96bad6d79bdd5a426b379e43fe26ad51035422356654d7f711b505


Unable to find image 'redmine:latest' locally
latest: Pulling from library/redmine
e4fff0779e6d: Pulling fs layer
861a67897794: Pulling fs layer
5e2ffead0346: Pulling fs layer
a8fa68e86ee9: Pulling fs layer
948e7b65e902: Pulling fs layer
4c52d81084b4: Pulling fs layer
bd87a8eecda1: Pulling fs layer
05379b07842c: Pulling fs layer
9591fac65863: Pulling fs layer
51e2c997aafe: Pulling fs layer
9ef0e51d9af2: Pulling fs layer
27a06a3e5d3a: Pulling fs layer
2adc65a96402: Pulling fs layer
05379b07842c: Waiting
9591fac65863: Waiting
51e2c997aafe: Waiting
9ef0e51d9af2: Waiting
27a06a3e5d3a: Waiting
2adc65a96402: Waiting
948e7b65e902: Waiting
bd87a8eecda1: Waiting
a8fa68e86ee9: Waiting
4c52d81084b4: Waiting
5e2ffead0346: Verifying Checksum
5e2ffead0346: Download complete
a8fa68e86ee9: Verifying Checksum
a8fa68e86ee9: Download complete
861a67897794: Verifying Checksum
861a67897794: Download complete
948e7b65e902: Download complete
4c52d81084b4: Verifying Checksum
4c52d81084b4: Download complete
e4

In [15]:
# http://localhost:8086

<img src="./image/d05-17.png" width="60%"></img>

## 레드마인 및 MariaDB 컨테이너 생성

MariaDB는 MySQL 개발자가 만든 만큼 MySQL과 비슷한 점이 아주 많다. 

<img src="./image/d05-18.png" width="30%"></img>

<img src="./image/d05-19.png" width="30%"></img>

네트워크 생성

In [16]:
!docker network create redmine000net3

b9426700b29c7d8dcc3427ce6a2ed165c3a5ba82d1751f7f7b0e3397e3a4adf8


MariaDB 컨테이너 생성 및 실행

In [17]:
!docker run --name mariadb000ex15 -dit \
  --net=redmine000net3 \
  -e MYSQL_ROOT_PASSWORD=mariarootpass \
  -e MYSQL_DATABASE=redmine000db   \
  -e MYSQL_USER=redmine000kun \
  -e MYSQL_PASSWORD=rkunpass \
  mariadb \
  --character-set-server=utf8mb4 \
  --collation-server=utf8mb4_unicode_ci \
  --default-authentication-plugin=mysql_native_password

1b523f03226ca96b8519de357f9e0362e45e2ea3ba52bbcd81caaea5f7068505


Unable to find image 'mariadb:latest' locally
latest: Pulling from library/mariadb
31e907dcc94a: Pulling fs layer
8687fa065e6d: Pulling fs layer
bc75b4546118: Pulling fs layer
90824338d93e: Pulling fs layer
c13aedba8d5d: Pulling fs layer
ad9066662cff: Pulling fs layer
537f82e52967: Pulling fs layer
a5e6bca88fae: Pulling fs layer
c13aedba8d5d: Waiting
ad9066662cff: Waiting
537f82e52967: Waiting
a5e6bca88fae: Waiting
90824338d93e: Waiting
8687fa065e6d: Verifying Checksum
8687fa065e6d: Download complete
bc75b4546118: Verifying Checksum
bc75b4546118: Download complete
90824338d93e: Verifying Checksum
90824338d93e: Download complete
c13aedba8d5d: Verifying Checksum
c13aedba8d5d: Download complete
537f82e52967: Verifying Checksum
537f82e52967: Download complete
a5e6bca88fae: Download complete
ad9066662cff: Verifying Checksum
ad9066662cff: Download complete
31e907dcc94a: Verifying Checksum
31e907dcc94a: Download complete
31e907dcc94a: Pull complete
8687fa065e6d: Pull complete
bc75b4546118: Pu

<img src="./image/d05-20.png" width="50%"></img>
<img src="./image/d05-21.png" width="50%"></img>

레드마인 컨테이너의 생성 및 실행

In [19]:
!docker run -dit --name redmine000ex16 \
  --network redmine000net3 -p 8087:3000 \
  -e REDMINE_DB_MYSQL=mariadb000ex15 \
  -e REDMINE_DB_DATABASE=redmine000db \
  -e REDMINE_DB_USERNAME=redmine000kun \
  -e REDMINE_DB_PASSWORD=rkunpass \
  redmine

367511e09b5e199d5f2dd0dced8d724f739b8b298cdd1b0fb120a5db7e304e86


<img src="./image/d05-22.png" width="50%"></img>

In [ ]:
# http://localhost:8087

#### 뒷정리

##### 컨테이너 뒷정리

In [5]:
# 컨테이너 목록 확인
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [6]:
# 컨테이너 종료
!docker stop edmine000ex16

Error response from daemon: No such container: edmine000ex16


In [1]:
# 컨테이너 삭제
!docker rm edmine000ex16

Error response from daemon: No such container: edmine000ex16


##### 이미지 뒷정리

In [2]:
# 이미지 목록 확인
!docker image ls

REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


In [27]:
# 이미지 삭제
!docker image rm docker/labs-vscode-installer:0.0.3

Error response from daemon: conflict: unable to remove repository reference "docker/labs-vscode-installer:0.0.3" (must force) - container cb078f6442ba is using its referenced image b5e14f161d00


##### 네트워크 뒷정리

In [1]:
# 네트워크 목록 확인
!docker network ls

NETWORK ID     NAME      DRIVER    SCOPE
9048b1acbcd7   bridge    bridge    local
db7534889888   host      host      local
3f3d252b73eb   none      null      local


In [14]:
# 네트워크 삭제
!docker network rm redmine000net2 redmine000net3

Error response from daemon: network redmine000net2 not found
Error response from daemon: network redmine000net3 not found


##### 볼륨 뒷정리

In [19]:
# 볼륨 목록 확인
!docker volume ls

DRIVER    VOLUME NAME


In [2]:
# 볼륨 삭제
!docker volume rm 2b6243c4bad7966c4306c9f8a84584af31eb2009f0f033ec8efed1f29debadfd

Error response from daemon: get 2b6243c4bad7966c4306c9f8a84584af31eb2009f0f033ec8efed1f29debadfd: no such volume
